In [2]:
import threading
import time
import requests
import json
import pandas as pd
import cx_Oracle
from datetime import datetime
import re

In [3]:
def get_request_url():
    getURL = 'http://openapi.seoul.go.kr:8088/595944524879756a3735434365634d/json/bikeList/1/1000'
    rentBikeStatus = requests.get(getURL)

    return rentBikeStatus.text

def get_parse_json(raw_json):
    all_data = []

    # 행
    for record in raw_json['rentBikeStatus']['row']:
        all_data.append(
            {"rackTotCnt": record.get("rackTotCnt") ,
             "stationName": record.get("stationName") ,
             "parkingBikeTotCnt": record.get("parkingBikeTotCnt") ,
             "shared": record.get("shared") ,
             "stationLatitude": record.get("stationLatitude") ,
             "stationLongitude": record.get("stationLongitude") ,
             "stationId": record.get("stationId"),
             }
        )
    return all_data

def json_to_df_info(raw_json):
    all_data = []
    column_list = ["rackTotCnt", "stationName", "parkingBikeTotCnt", "shared", "stationLatitude", "stationLongitude","stationId"]

    for record in raw_json['rentBikeStatus']['row']:
        all_data.append(
                [
                 record.get("rackTotCnt"), # 거치대개수
                 record.get("stationName"), # 대여소이름
                 record.get("parkingBikeTotCnt"), # 자전거주차총건수
                 record.get("shared"), # 거치율(대여 가능 자전거 수 / 거치대 수)
                 record.get("stationLatitude"), # 위도
                 record.get("stationLongitude"), # 경도
                 record.get("stationId"), # 대여소id
                ],
        )
        
    return column_list, all_data

In [11]:
def preprocessed_df_to_oracle(df):
    con = cx_Oracle.connect('bike/12345@192.168.0.78:1521/xe')
    cur = con.cursor()

    # '대여소이름', '거치대개수', '남은자전거수', '거치율', '위도', '경도', '대여소ID'
    sql_insert = '''
                insert into bike_real_time values(:STATIONNAME ,:RACKTOCNT ,:BIKETOCNT, :SHARED_RATE, :LATITUDE, :LONGITUDE, :STATIONID, :UPDATETIME)
                '''
    for i in range(len(df)):
        STATIONNAME = df.iloc[i]['STATIONNAME']
        STATIONNAME2 = STATIONNAME

        RACKTOCNT = df.iloc[i]['RACKTOCNT']
        RACKTOCNT2 = RACKTOCNT

        BIKETOCNT = df.iloc[i]['BIKETOCNT']
        BIKETOCNT2 = BIKETOCNT

        SHARED_RATE = df.iloc[i]['SHARED_RATE']
        SHARED_RATE2 = SHARED_RATE

        LAT = df.iloc[i]['LATITUDE']
        LON = df.iloc[i]['LONGITUDE']

        STATIONID = df.iloc[i]['STATIONID']

        UPDATETIME = df.iloc[i]['UPDATETIME']
        UPDATETIME2 = UPDATETIME

        cur.execute(sql_insert,
                    (STATIONNAME,RACKTOCNT,BIKETOCNT,SHARED_RATE,LAT,LON,STATIONID,UPDATETIME,)
                    )

    con.commit()
    cur.close()
    con.close()

def bike_real_time_info_collector():
    raw_str_json = get_request_url()

    if raw_str_json:
        raw_json = json.loads(raw_str_json)

        #parsed_json = get_parse_json(raw_json)

    column_list, all_data = json_to_df_info(raw_json)

    df = pd.DataFrame(all_data, columns = column_list)

    df = df[['stationName', 'rackTotCnt', 'parkingBikeTotCnt', 'shared', 'stationLatitude', 'stationLongitude',
                     'stationId']]
    df.columns = ['STATIONNAME', 'RACKTOCNT', 'BIKETOCNT', 'SHARED_RATE', 'LATITUDE', 'LONGITUDE', 'STATIONID']
    df['UPDATETIME'] = df['UPDATETIME'] = datetime.now()
    df['STATIONNAME'] = re.sub(r'[0-9]','',df['STATIONNAME'])
    df['STATIONNAME'].replace('.','').strip()
    
    #preprocessed_df_to_oracle(df)
    print(df)

In [5]:
# def bike_real_time_info_scheduler():
#     print('실시간 따릉이 정보 스케줄러 동작.\n')
#     while True:
#         bike_real_time_info_collector()
#         print("수집완료.")
#         time.sleep(60)

# def print_main_menu():
#     print('\n1. 따릉이 실시간 데이터 구축')
#     print('2. 스케줄러 종료')
#     print('* 엔터: 메뉴 업데이트\n')

# while True:
#     print_main_menu()
#     print('아래행에 메뉴입력: ')

#     selection = input()

#     if selection == '':
#         continue
#     else:
#         menu_num = int(selection)

#     if(menu_num == 1):
#         t = threading.Thread(target=bike_real_time_info_scheduler(), daemon=True)
#         t.start()
#     elif(menu_num == 2):
#         break
#     elif (menu_num == 0):
#         continue


1. 따릉이 실시간 데이터 구축
2. 스케줄러 종료
* 엔터: 메뉴 업데이트

아래행에 메뉴입력: 
1
실시간 따릉이 정보 스케줄러 동작.

           STATIONNAME RACKTOCNT BIKETOCNT SHARED_RATE     LATITUDE  \
0      102. 망원역 1번출구 앞        15        23         153  37.55564880   
1      103. 망원역 2번출구 앞        14        24         171  37.55495071   
2      104. 합정역 1번출구 앞        13         2          15  37.55062866   
3      105. 합정역 5번출구 앞         5         4          80  37.55000687   
4      106. 합정역 7번출구 앞        12        17         142  37.54864502   
..                 ...       ...       ...         ...          ...   
995     1405. 망우역 1번출구        10         1          10  37.59909821   
996     1406. 망우청소년수련관        10         3          30  37.60625076   
997   1407. 먹골역 1번출구 뒤        10        12         120  37.61151123   
998   1408. 먹골역 6번출구 앞        10         7          70  37.61072159   
999  1410. 면목 대원칸타빌아파트        10        11         110  37.57778168   

        LONGITUDE STATIONID                 UPDATETIME  
0    126.9

KeyboardInterrupt: 